In [63]:
import numpy as np
from copy import deepcopy

In [64]:
def filled(matrix):
    c = 0
    for i in matrix:
        for j in i:
            if j != 0:
                c+=1
    return c


def areIdentical(matrixOne, matrixTwo):
    # small conditions for the sake of complexity(code runs way faster now)
    # basically why to bother checking and comparing element of both if even filled are not same 
    if matrixOne[1][1] == matrixTwo[1][1] and filled(matrixOne) == filled(matrixTwo):
        return np.array_equal(matrixOne, matrixTwo) or np.array_equal(np.flipud(matrixOne), matrixTwo) or np.array_equal(np.fliplr(matrixOne), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne)), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne, 2), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne, 2)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne, 2)), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne,3), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne,3)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne,3)), matrixTwo)
    else:
        return False

def isWon(matrix):
    for i in range(3):
        if matrix[0][i] != 0 and matrix[0][i] == matrix[1][i] == matrix[2][i]:
            return matrix[0][i]
        if matrix[i][0] != 0 and matrix[i][0] == matrix[i][1] == matrix[i][2]:
            return matrix[i][0]
    if matrix[0][0] != 0 and matrix[0][0] == matrix[1][1] == matrix[2][2]:
        return matrix[0][0]
    if matrix[0][2] != 0 and matrix[0][2] == matrix[1][1] == matrix[2][0]:
        return matrix[0][2]
    return 0

def findEmptyPlaces(board):
    free = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                free.append(3*i+j)
    return free

def partialBot(matrix):
    for i in range(3):
        for j in range(3):
            # ROW CHECKING
            if matrix[i][j] == matrix[i][(j+1)%3] == 1 and matrix[i][(j+2)%3] == 0:
                return [i, (j+2)%3]
            # COLUMN CHECKING
            if matrix[i][j] == matrix[(i+1)%3][j] == 1 and matrix[(i+2)%3][j] == 0:
                return [(i+2)%3, j]
    for i in range(3):
        if matrix[i][i] == matrix[(i+1)%3][(i+1)%3] == 1 and matrix[(i+2)%3][(i+2)%3] == 0:
            return [(i+2)%3, (i+2)%3]
        if matrix[i][(2-i)%3] == matrix[(i+1)%3][(1-i)%3] == 1 and matrix[(i+2)%3][(-i)%3] == 0:
            return [(i+2)%3, -i%3]
        
    inp = np.random.choice(findEmptyPlaces(matrix))
    return [inp//3, inp%3]

partialBot([[0,0,0], [0,1,0], [1,0,0]])
    

[0, 2]

In [65]:
def getStates(matrix, isOurMove):
    moves = []
    for i in range(3):
        for j in range(3):
            if matrix[i][j] == 0:
                newMatrix = deepcopy(matrix)
                newMatrix[i][j] = 1 if isOurMove else 2
                moves.append(newMatrix)
    return moves

def createStates(matrix, isOurMove=True, states = []):
    if filled(matrix) == 9:
        return states
    newStates = getStates(matrix, isOurMove)
    for i in newStates:
        found = False
        for j in states:
            if areIdentical(i,j):
                found = True
                break
        if not found and not isWon(i) and filled(i) < 8:
            if isOurMove:
                states.append(i)
            createStates(i, isOurMove=not isOurMove, states = states)
    return states

In [66]:
# RUN FOR FIRST TIME AND STORE IN CACHE FOR NEXT TIMES(Finding 304 unique matrixes take time)
z = createStates(np.zeros((3,3)), False, [np.zeros((3,3))])
print('you\'re done')
%store z

you're done
Stored 'z' (list)


In [4]:
# RUN THIS FOR NEXT TIME SO YOU DON"T NEED TO DO COMPUTATION AGAIN
%store -r z

In [67]:
len(z) # 304!!! WE'RE ON TRACK!!!
# for i in z:
#     if filled(i) == 8:
#         displayGame(i)

304

In [68]:
"""
https://www.mscroggs.co.uk/blog/19
Initially, MENACE begins with four beads of each colour in the first move box, 
three in the third move boxes, two in the fifth move boxes and one in the final move boxes. 
Removing one bead from each box on losing means that later moves are more heavily discouraged. 
This helps MENACE learn more quickly, as the later moves are more likely to have led to the loss.
"""

def displayGame(board):
    # strange CLI asesthetics
    print('\n-------')
    for i in board:
        for j in i:
            if j == 1:
                print('O', end='|')
            elif j == 2:
                print('X', end='|')
            else:
                print('_', end='|')
        print('\n-------')

# putting intial matches in all. 4 of each unfilled place
def getInitalMatches(z):
    initialNumber = 4
    moves = list(range(9))
    allMatches = []
    for i in z:
        emptyMoves = []
        for _i in range(3):
            for _j in range(3):
                if i[_i][_j] == 0:
                    emptyMoves.append(_i * 3 + _j)
        allMatches.append({
            'matrix': i,
            'places': emptyMoves*initialNumber
        })
    return allMatches

def getMatchbox(board, matchBoxes):
    for i in matchBoxes:
        if areIdentical(board, i['matrix']):
            return i

initialMatches = getInitalMatches(z)
len(initialMatches)

304

In [41]:
np.random.choice(findEmptyPlaces([[0,1,0], [1,0,1], [0,0,1]]))


3

In [95]:
_initialMatches = deepcopy(initialMatches)
def trainMenace(iters, _initialMatches, showPrints, userInput = False):   
    for _ in range(iters):
        board = np.zeros((3,3))
        playerMove = False
        # array to keep track of menace moves so we can update at the end
        menaceMoves = []
        while filled(board) != 9 and not isWon(board):
            if playerMove:
                # player move logic
                inp = 0
                if userInput:
                    inp = int(input('your move:')) - 1 # well we start from 1 in head so....
                    board[inp//3][inp%3] = 2
                else:
                    x, y = partialBot(board)
                    board[x, y] = 2
                
            else:
                if filled(board) < 8:
                    # THE MAGIC PART LOL
                    # find appropriate matchbox from menace
                    matchBox = getMatchbox(board, _initialMatches)
                    if matchBox:
                        # SO WE WILL ROTATE/FLIP the matrix if needed to make it easier for menace to choose
                        board = deepcopy(matchBox['matrix'])
                        choice = np.random.choice(matchBox['places'])
                        menaceMoves.append({
                            'matrix': matchBox['matrix'],
                            'choice': choice
                        })
                        if showPrints:
                            print(f'menace played {choice+1}')
                        # updating the board
                        board[choice//3][choice%3] = 1
                else:
                    # automatically fill last choice for menace
                    for i in range(3):
                        for j in range(3):
                            if board[i][j] == 0:
                                if showPrints:
                                    print(f'menace played {i*3+j+1}')
                                board[i][j] = 1
            if showPrints:
                displayGame(board)
            playerMove = not playerMove

        winningState = isWon(board)
        if showPrints:
            if not winningState:
                print('DRAW SO MANACE WILL ADD 1 MATCH IN ALL')
            elif winningState == 1:
                print('MENACE WON SO WILL ADD 2 MATCH IN ALL')
            else:
                print('MENACE LOST SO WILL REMOVE 1 MATCH FROM ALL')
        # THE MOST MAGIC PART, CHANGING PROBABILITIES DEPENDING UPON WINNING LOSING AND ALL
        for i in menaceMoves:
            for j in _initialMatches:
                if np.array_equal(i['matrix'], j['matrix']):
                    if not winningState:
                        # if draw add 1
                        j['places'].append(i['choice'])
                    elif winningState == 1:
                        # if AI wins in just 5 moves(3 ai moves, 2 player moves, promote better)
                        if filled(board) == 5:
                            j['places'].extend([i['choice']]*4)
                        else:
                            # if won add twice
                            j['places'].extend([i['choice']]*2)
                    else:
                        # if lost remove once
                        j['places'].remove(i['choice'])
#                     print(j['places'])
                    break
    return _initialMatches
trained = trainMenace(1000, _initialMatches, False, False)
print('done training')

done training


In [ ]:
# to play with menace
trainMenace(1, trained, True, True)

In [100]:
# see posibilies for each move
for i in trained:
    m = i['places']
    d = {}
    displayGame(i['matrix'])
    for j in m:
#         d[j] = d.get(j, 0) + 1/len(m) # NUMBER
        d[j] = d.get(j, 0) + 1/len(m) # POSSIBILITIES
    print(d)


-------
_|_|_|
-------
_|_|_|
-------
_|_|_|
-------
{1: 0.0008090614886731392, 8: 0.004854368932038835, 6: 0.038834951456310676, 4: 0.3883495145631086, 2: 0.12944983818770245, 0: 0.10517799352750823, 3: 0.04773462783171518, 5: 0.18689320388349465, 7: 0.09789644012944994}

-------
X|O|_|
-------
_|_|_|
-------
_|_|_|
-------
{2: 0.13445378151260504, 4: 0.10084033613445377, 8: 0.3445378151260507, 3: 0.07563025210084033, 7: 0.01680672268907563, 5: 0.025210084033613446, 6: 0.3025210084033615}

-------
X|O|X|
-------
O|_|_|
-------
_|_|_|
-------
{4: 0.31818181818181823, 5: 0.2272727272727273, 8: 0.18181818181818182, 6: 0.13636363636363635, 7: 0.13636363636363635}

-------
X|O|X|
-------
O|X|O|
-------
_|_|_|
-------
{6: 0.46153846153846156, 8: 0.3076923076923077, 7: 0.23076923076923078}

-------
X|O|X|
-------
O|X|_|
-------
O|_|_|
-------
{8: 0.49999999999999994, 5: 0.3333333333333333, 7: 0.16666666666666666}

-------
X|O|X|
-------
O|X|_|
-------
_|O|_|
-------
{5: 0.42857142857142855,